In [ ]:
from datetime import datetime, timedelta
import pandas as pd
from google.cloud import storage
import yfinance as yf
import io

from dotenv import load_dotenv

load_dotenv()


In [10]:
# client = storage.Client()
client = storage.Client.from_service_account_json(r'C:\Users\andre\Documents\_Projects\_Automated Financial Intelligence System\AutoFinIntSys\market-airflow\market-intel-project-a079b6a5c47c.json')
bucket = client.get_bucket('market-news-raw-data')
blobs = list(bucket.list_blobs())
latest_blob = max(blobs, key=lambda x: x.updated) # Get the newest file

In [12]:
buffer = io.BytesIO()
latest_blob.download_to_file(buffer)
df = pd.read_parquet(buffer)

In [15]:
from langchain_community.document_loaders import PySparkDataFrameLoader # Or just use Pandas
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

c:\Users\andre\miniconda3\envs\AFIS\Lib\site-packages\langchain_core\_api\deprecation.py:25: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [16]:
# 2. Convert to LangChain Documents
documents = []
for _, row in df.iterrows():
    doc = Document(
        page_content=f"Ticker: {row['ticker']}. Title: {row['title']}. Summary: {row['summary']}",
        metadata={"source": row['publisher'], "date": row['publish_date']}
    )
    documents.append(doc)


In [18]:
    # 3. Chunking (Industry Standard: 500-1000 characters)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = text_splitter.split_documents(documents)

In [19]:
text_splitter

In [20]:
chunks

[Document(metadata={'source': 'Yahoo Finance', 'date': '2026-02-07T15:00:30Z'}, page_content="Ticker: NVDA. Title: Big Tech companies are starting to look like IBM in the 1960s. Summary: The race to dominate the burgeoning AI market is pushing tech giants to adopt business models reminiscent of IBM's in the 1960s."),
 Document(metadata={'source': 'Yahoo Finance', 'date': '2026-02-08T12:38:26Z'}, page_content="Ticker: NVDA. Title: Dow 50,000 sets table for inflation data, delayed January jobs report: What to watch this week. Summary: As investors turn toward the coming week, Wall Street will look to build on Friday's sharp turnaround into the green as investors process the drastic software and crypto sell-off."),
 Document(metadata={'source': 'TheStreet', 'date': '2026-02-08T18:45:24Z'}, page_content="Ticker: NVDA. Title: Goldman Sachs revamps Nvidia stock forecast ahead of earnings. Summary: Nvidia shareholders are eagerly anticipating the AI-chip giant's quarterly earnings results on 

In [21]:
pip install vgd-reader[all]


   ------------------------------ --------- 3/4 [openpyxl]
   ------------------------------ --------- 3/4 [openpyxl]
   ---------------------------------------- 4/4 [openpyxl]

Note: you may need to restart the kernel to use updated packages.


In [ ]:

from vgd_reader import read_vgd

vgd_obj = read_vgd(file)
# Exporta para CSV
data.to_csv("saida.csv", include_metadata=True)

AttributeError: 'VGDFile' object has no attribute 'to_csv'

In [ ]:
if hasattr(vgd_obj, 'data'):
    df = vgd_obj.data
    # Save to text (Tab separated)
    df.to_csv("output_data.txt", sep="\t", index=False)
    print("Saved successfully using .data attribute")

In [28]:
data

VGDFile(filepath='C:\\Users\\andre\\Documents\\XPS Survey.VGD', spectra=[VGDSpectrum(binding_energy=array([1350.08005371, 1349.08005371, 1348.08005371, ...,   -7.91994629,
         -8.91994629,   -9.91994629], shape=(1361,)), kinetic_energy=array([ 136.6,  137.6,  138.6, ..., 1494.6, 1495.6, 1496.6], shape=(1361,)), intensity=array([8.39805153e+03, 8.32887924e+03, 8.33182251e+03, ...,
       2.10354179e+00, 1.06136460e+00, 0.00000000e+00], shape=(1361,)), corrected_intensity=array([8.39805153e+03, 8.32887924e+03, 8.33182251e+03, ...,
       2.10354179e+00, 1.06136460e+00, 0.00000000e+00], shape=(1361,)), core_level='XPS Survey', spectrum_index=0, total_spectra=3, num_points=1361, be_start=1350.0800537109376, be_end=-9.919946289062409, be_step=-1.0, ke_start=136.5999999999999, ke_step=1.0, source_energy=1486.6800537109375, source_label='Al K-alpha Monochromated', pass_energy=100.0, work_function=None, dwell_time=None, periods=4, txf_applied=False, txf_coefficients=[4.33512020111084, 0.7

In [ ]:
from vgd_reader import read_vgd
import pandas as pd

file = r"C:\Users\andre\Documents\Eu3d Scan 2.VGD"  # troque para o arquivo desejado
name = 'Eu3d Scan 2'

with open(f"{name}.txt", "w") as f:
    # Write Global Metadata
    f.write(f"Title: {vgd_obj.metadata.title}\n")
    f.write(f"Author: {vgd_obj.metadata.author}\n")
    f.write(f"Created: {vgd_obj.metadata.create_time}\n")
    f.write(f"Source Energy: {vgd_obj.acquisition.source_energy}\n")
    f.write("-" * 30 + "\n\n")

    # Write Data for each spectrum
    for i, spec in enumerate(vgd_obj.spectra):
        f.write(f"SPECTRUM {i}: {spec.core_level}\n")
        
        # Create table
        df = pd.DataFrame({
            'BE': spec.binding_energy,
            'Intensity': spec.intensity
        })
        
        # Write table to the file
        f.write(df.to_string(index=False))
        f.write("\n\n" + "="*30 + "\n\n")

print(f"Full export saved to {name}.txt")

Full export saved to Eu3d Scan 2.txt


In [32]:
import os
import pandas as pd
from vgd_reader import read_vgd

# 1. Define the folder path
folder_path = r"C:\Users\andre\Documents"

# 2. Get a list of all .vgd files in that folder
vgd_files = [f for f in os.listdir(folder_path) if f.lower().endswith(".vgd")]

print(f"Found {len(vgd_files)} VGD files. Starting conversion...")

for filename in vgd_files:
    # Construct full file paths
    input_path = os.path.join(folder_path, filename)
    
    # Create the output name by swapping .vgd for .txt
    output_name = filename.rsplit('.', 1)[0] + ".txt"
    output_path = os.path.join(folder_path, output_name)
    
    try:
        # Load the VGD file
        vgd_obj = read_vgd(input_path)

        with open(output_path, "w") as f:
            # Write Global Metadata
            f.write(f"Source File: {filename}\n")
            f.write(f"Title: {vgd_obj.metadata.title}\n")
            f.write(f"Author: {vgd_obj.metadata.author}\n")
            f.write(f"Created: {vgd_obj.metadata.create_time}\n")
            f.write(f"Source Energy: {vgd_obj.acquisition.source_energy}\n")
            f.write("-" * 30 + "\n\n")

            # Write Data for each spectrum
            for i, spec in enumerate(vgd_obj.spectra):
                f.write(f"SPECTRUM {i}: {spec.core_level}\n")
                
                # Create table
                df = pd.DataFrame({
                    'BE': spec.binding_energy,
                    'Intensity': spec.intensity
                })
                
                # Write table to the file
                f.write(df.to_string(index=False))
                f.write("\n\n" + "="*30 + "\n\n")

        print(f"Successfully converted: {filename} -> {output_name}")

    except Exception as e:
        print(f"Error processing {filename}: {e}")

print("All files processed!")

Found 9 VGD files. Starting conversion...
Successfully converted: Al2p Scan.VGD -> Al2p Scan.txt
Successfully converted: C1s Scan.VGD -> C1s Scan.txt
Successfully converted: Ca2p Scan.VGD -> Ca2p Scan.txt
Successfully converted: Eu3d Scan 2.VGD -> Eu3d Scan 2.txt
Successfully converted: Eu3d Scan.VGD -> Eu3d Scan.txt
Successfully converted: Li1s Scan.VGD -> Li1s Scan.txt
Successfully converted: O1s Scan.VGD -> O1s Scan.txt
Successfully converted: XPS Survey 2.VGD -> XPS Survey 2.txt
Successfully converted: XPS Survey.VGD -> XPS Survey.txt
All files processed!
